In [1]:
from model_utils import load_model_and_tokenizer, compute_cross_entropy
import torch

/home/zijian/miniconda3/envs/llm/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/zijian/miniconda3/envs/llm/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
model_path = "../llm/Llama-2-7b-chat-hf"
device = "cuda"

In [3]:
# Load the model and tokenizer
model, tokenizer = load_model_and_tokenizer(model_path, low_cpu_mem_usage=True, 
                       use_cache=False,
                       device=device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
qn = """
What is 1 + 1 + (-1) + 1 + 1 + 1?
"""
input_ids = tokenizer.encode(qn, return_tensors="pt").to(device)

In [26]:
compute_cross_entropy(input_ids, model, max_new_tokens=500, do_sample=False, num_return_sequences=1)

tensor(0.1811, device='cuda:0')

In [31]:
res = model.generate(input_ids, max_length=500, num_return_sequences=1, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(tokenizer.decode(res[0], skip_special_tokens=True))


What is 1 + 1 + (-1) + 1 + 1 + 1?

Using the order of operations (PEMDAS), evaluate the expression:

1 + 1 + (-1) + 1 + 1 + 1

The expression can be evaluated as follows:

1 + 1 = 2
2 + (-1) = 1
1 + 1 = 2
2 + 1 = 3
3 + 1 = 4
4 + 1 = 5

Therefore, 1 + 1 + (-1) + 1 + 1 + 1 = 5.


In [7]:
output = model.generate(input_ids, return_dict_in_generate=True, output_scores=True, max_length=500, num_return_sequences=1, do_sample=False)
transition_scores = model.compute_transition_scores(output.sequences, output.scores, normalize_logits=True)

In [8]:
idx = torch.exp(transition_scores) < 1-1e-3

transition_scores = transition_scores[idx]

In [9]:
-torch.mean(transition_scores * torch.exp(transition_scores))

tensor(0.1020, device='cuda:0')